<a href="https://colab.research.google.com/github/jsqzhu/BikeShareTO-ActiveAI/blob/main/01_BikeShare_EDA_Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# File saved on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import io
os.chdir("/content/drive/MyDrive/Colab Notebooks/Data/")

In [3]:
import json
from dateutil.parser import parse
import pandas as pd
import numpy as np
import requests
import sys

#from parse_names import parseName, lookup
#from data_exceptions import missing_coordinates, location_rename, unknown_locations

# Adding Weather information to usage data with distance

## Functions

In [4]:
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%y-%m-%d %H:%M')
dateparse_timezone = lambda x: datetime.strptime(x[:-4], '%Y-%m-%d %H:%M:%S')

In [5]:
from datetime import datetime, timedelta
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

## Load Datasets and standardize date formats

In [6]:
## load combined data with distance
#os.listdir()

df = pd.read_csv('final_trip_data_with_distance_duration.csv', index_col=[0], parse_dates=['Start Time'], date_parser=dateparse)

In [7]:
df.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,orig_coords,dest_coords,Origin Address,Destination Address,Duration (s),Distance (m)
0,7334128,648,7003,2020-01-01 00:08:00,Madison Ave / Bloor St W,7271,20-01-01 0:19,Yonge St / Alexander St - SMART,3104,Annual Member,"(43.667158, -79.402761)","(43.662862, -79.383572)","Madison Ave / Bloor St W, Toronto, ON M5R 2S1,...","Yonge St / Alexander St - SMART, Toronto, ON M...",595,2312
1,7350985,799,7003,2020-01-06 08:34:00,Madison Ave / Bloor St W,7271,20-01-06 8:47,Yonge St / Alexander St - SMART,4232,Annual Member,"(43.667158, -79.402761)","(43.662862, -79.383572)","Madison Ave / Bloor St W, Toronto, ON M5R 2S1,...","Yonge St / Alexander St - SMART, Toronto, ON M...",595,2312
2,7405155,745,7003,2020-01-20 09:12:00,Madison Ave / Bloor St W,7271,20-01-20 9:24,Yonge St / Alexander St - SMART,3465,Annual Member,"(43.667158, -79.402761)","(43.662862, -79.383572)","Madison Ave / Bloor St W, Toronto, ON M5R 2S1,...","Yonge St / Alexander St - SMART, Toronto, ON M...",595,2312
3,7453581,689,7003,2020-02-01 12:19:00,Madison Ave / Bloor St W,7271,20-02-01 12:31,Yonge St / Alexander St - SMART,4279,Annual Member,"(43.667158, -79.402761)","(43.662862, -79.383572)","Madison Ave / Bloor St W, Toronto, ON M5R 2S1,...","Yonge St / Alexander St - SMART, Toronto, ON M...",595,2312
4,7471316,606,7003,2020-02-05 14:52:00,Madison Ave / Bloor St W,7271,20-02-05 15:02,Yonge St / Alexander St - SMART,1645,Annual Member,"(43.667158, -79.402761)","(43.662862, -79.383572)","Madison Ave / Bloor St W, Toronto, ON M5R 2S1,...","Yonge St / Alexander St - SMART, Toronto, ON M...",595,2312


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 945169 entries, 0 to 945168
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Trip Id              945169 non-null  int64         
 1   Trip  Duration       945169 non-null  int64         
 2   Start Station Id     945169 non-null  int64         
 3   Start Time           945169 non-null  datetime64[ns]
 4   Start Station Name   945169 non-null  object        
 5   End Station Id       945169 non-null  int64         
 6   End Time             945169 non-null  object        
 7   End Station Name     945169 non-null  object        
 8   Bike Id              945169 non-null  int64         
 9   User Type            945169 non-null  object        
 10  orig_coords          945169 non-null  object        
 11  dest_coords          945169 non-null  object        
 12  Origin Address       945169 non-null  object        
 13  Destination Ad

In [9]:
#hour_rounder(df.iloc[0,3])
df['Rounded_Start_Time'] = df['Start Time'].apply(lambda x: hour_rounder(x))

In [18]:
## add weather data for each trip
weather = pd.read_csv('https://raw.githubusercontent.com/jsqzhu/BikeShareTO-ActiveAI/main/Raw_Data/weatherstats_toronto_hourly.csv', parse_dates=['date_time_local'], date_parser=dateparse_timezone)
weather.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,date_time_local,unixtime,pressure_station,pressure_sea,wind_dir,wind_dir_10s,wind_speed,wind_gust,relative_humidity,dew_point,...,windchill,humidex,visibility,health_index,cloud_cover_4,cloud_cover_8,cloud_cover_10,solar_radiation,max_air_temp_pst1hr,min_air_temp_pst1hr
0,2022-03-06 13:00:00,1646589600,98.26,100.35,WSW,25.0,52.0,79.0,58.0,7.2,...,NaN,NaN,24100.0,NaN,NaN,4.0,NaN,NaN,16.0,15.0
1,2022-03-06 12:00:00,1646586000,98.22,100.31,WSW,24.0,48.0,78.0,60.0,7.7,...,NaN,NaN,24100.0,2.6,NaN,6.0,NaN,NaN,15.7,14.9
2,2022-03-06 11:00:00,1646582400,98.18,100.26,WSW,24.0,47.0,85.0,65.0,8.5,...,NaN,NaN,24100.0,2.6,NaN,5.0,NaN,NaN,15.4,14.6
3,2022-03-06 10:00:00,1646578800,98.16,100.24,SW,23.0,56.0,69.0,63.0,7.8,...,NaN,NaN,24100.0,2.6,NaN,3.0,NaN,NaN,14.7,13.6
4,2022-03-06 09:00:00,1646575200,98.25,100.34,SW,22.0,41.0,60.0,76.0,9.6,...,NaN,NaN,24100.0,2.9,NaN,2.0,NaN,NaN,13.7,11.9


## Add Features from Weather data

In [19]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 0 to 100000
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   date_time_local      100001 non-null  datetime64[ns]
 1   unixtime             100001 non-null  int64         
 2   pressure_station     99970 non-null   float64       
 3   pressure_sea         99970 non-null   float64       
 4   wind_dir             77579 non-null   object        
 5   wind_dir_10s         99135 non-null   float64       
 6   wind_speed           99972 non-null   float64       
 7   wind_gust            17794 non-null   float64       
 8   relative_humidity    99952 non-null   float64       
 9   dew_point            99957 non-null   float64       
 10  temperature          99965 non-null   float64       
 11  windchill            22911 non-null   float64       
 12  humidex              16372 non-null   float64       
 13  visibility    

In [20]:
cols_to_add = ['date_time_local','wind_speed', 'temperature','windchill','visibility']
df_combined = pd.merge(df, weather[cols_to_add], how='left',left_on='Rounded_Start_Time', right_on='date_time_local')

In [21]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 945169 entries, 0 to 945168
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Trip Id              945169 non-null  int64         
 1   Trip  Duration       945169 non-null  int64         
 2   Start Station Id     945169 non-null  int64         
 3   Start Time           945169 non-null  datetime64[ns]
 4   Start Station Name   945169 non-null  object        
 5   End Station Id       945169 non-null  int64         
 6   End Time             945169 non-null  object        
 7   End Station Name     945169 non-null  object        
 8   Bike Id              945169 non-null  int64         
 9   User Type            945169 non-null  object        
 10  orig_coords          945169 non-null  object        
 11  dest_coords          945169 non-null  object        
 12  Origin Address       945169 non-null  object        
 13  Destination Ad

In [22]:
df_combined[df_combined['date_time_local'].isnull()]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,...,Origin Address,Destination Address,Duration (s),Distance (m),Rounded_Start_Time,date_time_local,wind_speed,temperature,windchill,visibility


# Original dataset

In [ ]:
#usage_data=pd.read_csv('https://raw.githubusercontent.com/jsqzhu/BikeSharbeTO-ActiveAI/main/Raw_Data/combined-data.csv')
usage_data=pd.read_csv('https://media.githubusercontent.com/media/jsqzhu/BikeShareTO-ActiveAI/main/Raw_Data/combined-data.csv', encoding='ISO-8859-1')

In [ ]:
usage_data.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,7334128,648,7003,20-01-01 0:08,Madison Ave / Bloor St W,7271.0,20-01-01 0:19,Yonge St / Alexander St - SMART,3104,Annual Member
1,7334129,419,7007,20-01-01 0:10,College St / Huron St,7163.0,20-01-01 0:17,Yonge St / Wood St,2126,Annual Member
2,7334130,566,7113,20-01-01 0:13,Parliament St / Aberdeen Ave,7108.0,20-01-01 0:22,Front St E / Cherry St,4425,Annual Member
3,7334131,1274,7333,20-01-01 0:17,King St E / Victoria St,7311.0,20-01-01 0:38,Sherbourne St / Isabella St,4233,Annual Member
4,7334132,906,7009,20-01-01 0:19,King St E / Jarvis St,7004.0,20-01-01 0:34,University Ave / Elm St,2341,Casual Member


In [ ]:
usage_data = pd.read_csv('https://raw.githubusercontent.com/jsqzhu/BikeShareTO-ActiveAI/main/Raw_Data/2021-01.csv', encoding="ISO-8859-1")

In [ ]:
usage_data.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,10644218,1315,7021,01/01/2021 00:04,Bay St / Albert St,7164.0,01/01/2021 00:26,Gould St / Yonge St (Ryerson University),6795.0,Annual Member
1,10644220,396,7534,01/01/2021 00:07,Walnut Ave / Queen St W,7524.0,01/01/2021 00:13,Lisgar Park,4176.0,Casual Member
2,10644221,86,7162,01/01/2021 00:10,Hayter St / Laplante Ave,7006.0,01/01/2021 00:11,Bay St / College St (East Side),1814.0,Annual Member
3,10644222,741,7003,01/01/2021 00:10,Madison Ave / Bloor St W,7272.0,01/01/2021 00:22,Yonge St / Dundonald St - SMART,198.0,Casual Member
4,10644223,2073,7562,01/01/2021 00:11,Priscilla Ave / Dundas St W - SMART,7562.0,01/01/2021 00:45,Priscilla Ave / Dundas St W - SMART,6688.0,Casual Member


# Station info

In [ ]:
# json file: https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information
url='https://raw.githubusercontent.com/jsqzhu/BikeShareTO-ActiveAI/main/station_information.json'
resp = requests.get(url)
station_info = json.loads(resp.text)

In [ ]:
station_info_df = pd.json_normalize(station_info, record_path=['data','stations'])

In [ ]:
station_info_df.tail()

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,rental_methods,groups,obcn,nearby_distance,post_code,cross_street
612,7686,Lower Jarvis / The Esplanade,REGULAR,43.647821,-79.370472,0.0,The Esplanade & Lower Jarvis St,15,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",[],,1000.0,M5E 1Z2,The Esplanade & Lower Jarvis St
613,7687,Bloor / Gladstone - SMART,SMARTMAPFRAME,43.660202,-79.434273,0.0,"1101 Bloor Street West, Toronto",12,"[KEY, TRANSITCARD, PHONE]",[],,1000.0,M6H 1M8,Bloor / Gladstone
614,7688,High Park Blvd / Parkside Dr,REGULAR,43.644887,-79.456412,0.0,200 Parkside Drive,15,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",[],,1000.0,M6R 0A1,High Park Blvd / Parkside Dr
615,7689,20 Eaton,ELECTRICBIKESTATION,43.680012,-79.344033,0.0,26 Lipton Avenue,26,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",[],,1000.0,M4J 1N9,NaN
616,7691,Christie St / Dupont St,REGULAR,43.671656,-79.420727,0.0,Christie St / Dupont St,19,"[KEY, TRANSITCARD, CREDITCARD, PHONE]",[],,1000.0,M6G 1Z4,Christie St / Dupont St


In [ ]:
data=pd.read_csv('https://raw.githubusercontent.com/jsqzhu/BikeShareTO-ActiveAI/main/Raw_Data/2021-01.csv', encoding="ISO-8859-1")

In [ ]:
data.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,10644218,1315,7021,01/01/2021 00:04,Bay St / Albert St,7164.0,01/01/2021 00:26,Gould St / Yonge St (Ryerson University),6795.0,Annual Member
1,10644220,396,7534,01/01/2021 00:07,Walnut Ave / Queen St W,7524.0,01/01/2021 00:13,Lisgar Park,4176.0,Casual Member
2,10644221,86,7162,01/01/2021 00:10,Hayter St / Laplante Ave,7006.0,01/01/2021 00:11,Bay St / College St (East Side),1814.0,Annual Member
3,10644222,741,7003,01/01/2021 00:10,Madison Ave / Bloor St W,7272.0,01/01/2021 00:22,Yonge St / Dundonald St - SMART,198.0,Casual Member
4,10644223,2073,7562,01/01/2021 00:11,Priscilla Ave / Dundas St W - SMART,7562.0,01/01/2021 00:45,Priscilla Ave / Dundas St W - SMART,6688.0,Casual Member
